In [6]:
import numpy as np
import xgboost as xgb
import pandas as pd
import category_encoders as ce
import sklearn

#TODO: NOT REALLY WORKING

Data loading

In [10]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

train_X = train_data.iloc[:,:-1]
train_y = train_data.iloc[:,-1]
test_X = test_data.iloc[:,:]
test_y = test_data.iloc[:,0]

#Remove redundant data
# TODO: decide on which data to keep and which to drop
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

#Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)

#TODO: Select which data should be encoded
ce_OHE = ce.OneHotEncoder(cols=['loan_status', 'application_type', 'home_ownership', 'verification_status'])
train_data = ce_OHE.fit_transform(train_data)

ce_BE = ce.BinaryEncoder(cols=['loan_status'])
train_data = ce_BE.fit_transform(train_data)

train_X = train_data.iloc[:,:-1]
train_y = train_data.iloc[:,-1]
test_X = test_data.iloc[:,:]

#Replace subgrade with numerical value
ce_LOO = ce.LeaveOneOutEncoder(cols=['grade', 'emp_title', 'issue_d', 'addr_state', 'purpose', 'initial_list_status', 'title', 'application_type', 'home_ownership', 'verification_status', 'sub_grade', 'zip_code', 'emp_length', 'mort_acc', 'earliest_cr_line'])
train_X = ce_LOO.fit_transform(train_X, train_y)

# train_X.is_null().sum()

/Users/Rohan/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/Rohan/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [7]:
data_dmatrix = xgb.DMatrix(data=train_X, label=train_y)

/Users/jarroyo/miniforge3/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [21]:
train_X, val_X, train_y, val_y = sklearn.model_selection.train_test_split(train_X, train_y, test_size=0.2, random_state=0)

In [11]:
parameters = {"objective": "binary:logistic",
              "max_depth": 4,
              "lambda": 10,
              "learning_rate": 0.01,
              "n_estimators": 1}

xgb_clf = xgb.XGBClassifier(**parameters)
xgb_clf.fit(train_X, train_y)
pred_y = xgb_clf.predict(val_X)
score = sklearn.metrics.roc_auc_score(val_y, pred_y)
print(score)


/Users/Rohan/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:23:04] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1.0


In [26]:
test_X = test_data.iloc[:,:]
test_preds = xgb_clf.predict(test_X)
print(test_preds)

ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:int_rate, grade, sub_grade, emp_title, emp_length, home_ownership, verification_status, issue_d, purpose, title, zip_code, addr_state, earliest_cr_line, revol_util, initial_list_status, application_type